In [1]:
# initialize
import mne
import os
import scipy.io
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from scipy import signal
from scipy import stats
import pandas as pd
import pickle
import warnings
warnings.filterwarnings('ignore')
from itertools import permutations,combinations
from IPython.display import clear_output
import seaborn as sns
from scipy.linalg import toeplitz
from numpy import linalg as LA
from mne.event import define_target_events
from operator import itemgetter 

data_path = os.path.dirname(os.path.dirname(os.getcwd()))


save_path = data_path + '/python/data/coherence'
info = mne.io.read_raw_fif((save_path+'-info'),preload=True)

subject_name = ['Alice','Andrea','Daniel','Elena','Elenora','Elisa','Federica','Francesca','Gianluca1','Giada','Giorgia',
                'Jonluca','Laura','Leonardo','Linda','Lucrezia','Manu','Marco','Martina','Pagani','Pasquale','Sara',
                'Silvia','Silvia2','Tommaso']


save_path = data_path + '/python/data/extracted_features/features.pkl'
feat = pd.read_pickle(save_path)


clear_output()

In [2]:
# data selection functions
def get_eeg(mat,resample_freq):
    trial = mat['trialdata_noartifacts_ica']['trial'][0][0][0]
    time = mat['trialdata_noartifacts_ica']['time'][0][0][0]
    label = mat['trialdata_noartifacts_ica']['label'][0][0]
    trialinfo = mat['trialdata_noartifacts_ica']['trialinfo'][0][0]

    B=[]
    for i in range(0,trialinfo.shape[0]):
            c = trial[i][0:59,:]
            c = scipy.signal.decimate(c,resample_freq,axis=1)
            t = scipy.signal.decimate(time[i][0], resample_freq,axis=0)
            idx = np.where(mat['originalInfo']['trialinfo'][0][0][:,0]==trialinfo[i][0])[0][0]
            df = pd.DataFrame({'trialno':trialinfo[i][0],'eeg':[c],
                               'eegTime':[np.round(t,decimals=4)],
                               'filename':mat['originalInfo']['filename'][0][0][idx][0][0]})
            
            if(t[-1]>0):
                B.append(df)
                print(i,end=' ')
    
    A = pd.concat((B),axis=0)
    A.trialno = A.trialno.astype(int)
    clear_output()
    return A

def get_EMA(mat,feat):    
    #trialno = mat['experiment']['media'][0,0]['permute'][0][0][0]    
    response = np.stack(mat['experiment']['media'][0,0]['Cresponse'][0][0].flatten()) - \
                np.stack(mat['experiment']['media'][0,0]['Sresponse'][0][0].flatten())    
    RT = np.stack(mat['experiment']['media'][0,0]['responseT'][0][0].flatten())   
    filename = np.stack(mat['experiment']['media'][0][0]['filename'][0][0][0]).flatten()
    filename = [w.replace('.wav', '') for w in filename]
    
    df1 = pd.DataFrame({'trialno': range(200)})
    #df1['trialno'] = trialno
    df1['response'] = response
    df1['RT'] = RT
    df1['filename'] = filename
    
    A =[]
    for i in range(0,df1.shape[0]):        
        aa = feat[feat['filename']==df1.iloc[i]['filename']]
        aa = pd.concat([df1.iloc[i],aa.iloc[0].drop(['filename','trialno'])])
        aa = pd.DataFrame(aa)
        A.append(aa.T)
    A = pd.concat((A),axis=0)
    A['trialno'] = A['trialno']+1
    A.trialno = A.trialno.astype(int)
    return A

In [3]:
resample_freq=100


In [4]:
# downsample and save eeg for each subject
for s in range(0,len(subject_name)):
    a = os.path.dirname(os.path.dirname(os.getcwd()))
    raw_fname = a +'/data/cleaned_eeg/artifactcleanEEG/'+subject_name[s]+'_artifact_clean.mat'
    mat = scipy.io.loadmat(raw_fname)
    eeg = get_eeg(mat,int(1000/resample_freq))
    
    a = os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))
    raw_fname = a +'/exp/data/matlab_exp_data/'+subject_name[s]+'.mat'
    mat = scipy.io.loadmat(raw_fname)
    ema = get_EMA(mat,feat)
    ema =  ema.drop(['filename'],axis=1)
    A = eeg.merge(ema,on='trialno')
    
    #take only the correct response
    B = A[A['response']==0]
    B.to_pickle(data_path + '/python/data/preprocessed/'+subject_name[s]+'_eegEMAdownsampled_'
                +str(resample_freq)+'.pkl')


In [5]:
B.head(2)

,trialno,eeg,eegTime,filename,response,RT,condition,envelop,phonemeRate,phonemeDuration,...,Trial_len,jawaopening,lipaparature,lipProtrusion,TBCD,TMCD,TTCD,TB_ui,TM_ui,TT_ui
0,1,"[[8.618028885153109, 3.662948080810434, -4.204...","[-0.6922, -0.6821, -0.6723, -0.6624, -0.6524, ...",olm_24_1,0,1660,1,"[3.832434607550665e-05, 0.00013719255743702754...",12.234,0.0817847,...,7.52,"[33.38580739483703, 39.676152360506904, 37.567...","[17.610024474474002, 25.107035518724445, 22.70...","[4.604347144454266, 6.606904150135845, 5.97323...","[13.23606289523633, 21.67966276935423, 19.1554...","[18.610907879714976, 26.011312829994868, 23.98...","[16.98895509593703, 26.913543536374142, 22.603...","[42.53315150740693, 48.696668773481456, 46.574...","[32.59023269766158, 40.84762190423706, 38.0411...","[25.717550074686077, 35.86752127587399, 31.489..."
1,2,"[[-11.597592772674659, -7.952548743322001, -3....","[-0.6922, -0.6821, -0.6723, -0.6624, -0.6524, ...",olm_55_7,0,929,3,"[-2.5637810971338272e-05, 0.000178533399220979...",14.0187,0.0714158,...,2.14,"[26.86345073282078, 28.128479636442137, 27.358...","[9.629222866649158, 11.176926546698468, 10.163...","[2.6409088613070084, 2.8861672723729725, 2.680...","[4.282308388817926, 4.937723851772933, 4.51572...","[5.643807511839662, 6.8842184810520655, 6.2689...","[6.490858629357461, 9.48510606586409, 8.257436...","[31.70978523867261, 31.00262974355369, 31.4080...","[21.614265139989854, 21.42950538191492, 21.595...","[14.985003723408758, 18.151110449369455, 16.96..."


# for coherence measurement in fildtrip

In [6]:
trial_len = 2 # second
remove_first = 0.5 # second
delay = np.arange(-5,5.25,0.25) / 10
features = ['envelop','lipaparature']

In [7]:
# export to matlab filedtrip format

D = np.round(abs(delay *resample_freq),decimals=0)

for s in range(0,len(subject_name)):
    save_path = data_path + '/python/data/preprocessed/'+subject_name[s]+'_eegEMAdownsampled_'\
                +str(resample_freq)+'.pkl'
    data = pd.read_pickle(save_path)
    
    EEG = []
    EMA = []
    A = []
    for i in range(0,data.shape[0]):
        t = np.argmin(abs(data.iloc[i]['eegTime'] - 0))
        eeg = data.iloc[i]['eeg'][:,t:]
        ema = np.stack(data.iloc[i][features].get_values())

        if(eeg.shape[1]>ema.shape[1]):        
            ema = np.pad(ema, ((0,0),(0,eeg.shape[1]-ema.shape[1])), 'constant')   
        elif(ema.shape[1]>eeg.shape[1]):
            eeg = np.pad(eeg, ((0,0),(0,ema.shape[1]-eeg.shape[1])), 'constant')

        EEG.append(eeg)
        EMA.append(ema)
        A.append(eeg.shape[1])
        
    B = np.zeros((data.shape[0],59,max(A)))
    C = np.zeros((data.shape[0],len(features),max(A)))
    for i in range(0,B.shape[0]):
        B[i,:,:EEG[i].shape[1]] = EEG[i] 
        C[i,:,:EMA[i].shape[1]] = EMA[i]
        

    # all data
    EEG = B
    EMA = C
    
    # remove first 
    EEG = EEG[:,:,int(remove_first*100):]
    EMA = EMA[:,:,int(remove_first*100):]
    
    
    for d in range(0,len(delay)):
        t = int(D[d])
        if(delay[d]<0):
            ema = EMA[:,:,t:]
            eeg = EEG[:,:,:-t]
        elif(delay[d]==0):
            ema = EMA
            eeg = EEG
        else:
            ema = EMA[:,:,:-t]
            eeg = EEG[:,:,t:]

        A = np.concatenate((eeg,ema),axis=1)   
        
        if(trial_len*resample_freq<=A.shape[2]):
            A = A[:,:,:trial_len*resample_freq+1]
            
            save_path = data_path + '/python/data/coherence_analysis_matlab/'+subject_name[s]+\
                    '-trialLength'+str(trial_len)+'-delay'+str(delay[d])+'.mat'
            scipy.io.savemat(save_path, {'data':A,'label':np.stack(info.ch_names)})
        else:
            print('---error-----trial length is bigger')
            
        
